## Mesh (.stl) to Point Cloud

1. Uniform sampling of 4096 points 
2. Zero Centering 
3. Unit Scaling 
4. Feature preservation

**Output**: 
1. Points: 4096 X 3 
2. Centroid: Original (x, y, z) center Point of car before moving 
3. Scale: Single Numerical value used to represent the original size, distance of centroid from farthest point 


In [16]:
import os
import glob
import numpy as np
import open3d as o3d
from tqdm import tqdm

def mesh_to_pointcloud(stl_path, n_samples=4056):
    """
    Converts an STL mesh to a normalized point cloud using 
    Surface Sampling followed by Farthest Point Sampling (FPS).
    """
    # 1. Load Mesh
    mesh = o3d.io.read_triangle_mesh(stl_path)
    
    # 2. Pre-process Mesh (Essential for DrivAerNet++ STLs)
    mesh.remove_degenerate_triangles()
    mesh.remove_duplicated_vertices()
    
    # 3. Surface Sampling 
    # We sample 5x more points than needed initially to give FPS a good 'pool' to pick from
    temp_pcd = mesh.sample_points_uniformly(number_of_points=n_samples * 5)
    pts = np.asarray(temp_pcd.points).astype(np.float32)
    
    # 4. Normalize (Center and Scale to Unit Sphere)
    centroid = np.mean(pts, axis=0)
    pts -= centroid
    scale = np.max(np.linalg.norm(pts, axis=1))
    pts /= (scale + 1e-9)
    
    # 5. Farthest Point Sampling (FPS) 
    # This ensures the 2048 points are spread perfectly over the car's geometry
    fps_indices = farthest_point_sample_numpy(pts, n_samples)
    final_pts = pts[fps_indices]
    
    return final_pts, centroid, scale

def farthest_point_sample_numpy(points, n_samples):
    """ Standard FPS implementation """
    N = points.shape[0]
    centroids = np.zeros((n_samples,), dtype=np.int64)
    distances = np.ones((N,), dtype=np.float32) * 1e10
    farthest = np.random.randint(0, N)

    for i in range(n_samples):
        centroids[i] = farthest
        centroid = points[farthest]
        dist = np.sum((points - centroid) ** 2, axis=1)
        mask = dist < distances
        distances[mask] = dist[mask]
        farthest = np.argmax(distances)
    return centroids

def process_dataset(input_dir, output_dir, n_points=2048):
    os.makedirs(output_dir, exist_ok=True)
    stl_files = sorted(glob.glob(os.path.join(input_dir, "*.stl")))
    
    print(f"Found {len(stl_files)} STL files. Starting conversion...")
    
    for stl_path in tqdm(stl_files):
        fname = os.path.splitext(os.path.basename(stl_path))[0]
        out_path = os.path.join(output_dir, f"{fname}.npz")
        
        if os.path.exists(out_path): continue
        
        try:
            pts, center, scale = mesh_to_pointcloud(stl_path, n_samples=n_points)
            
            # Save as compressed numpy for PointNet training
            np.savez(out_path, points=pts, centroid=center, scale=scale)
        except Exception as e:
            print(f"Error processing {fname}: {e}")

if __name__ == "__main__":
    # Update these paths to your DrivAerNet++ folders
    INPUT_FOLDER = "02_dataset/car_design_mesh/F_D_WM_WW_1_subset_01"
    OUTPUT_FOLDER = "02_dataset/car_design_mesh/pc_processed_2"
    
    process_dataset(INPUT_FOLDER, OUTPUT_FOLDER, n_points=4096)

Found 202 STL files. Starting conversion...


100%|██████████| 202/202 [11:18<00:00,  3.36s/it]


In [8]:
import numpy as np
import open3d as o3d
import os

def npz_to_ply(npz_path, output_ply_path, denormalize=True):
    """
    Loads a processed .npz file and saves it as a .ply point cloud.
    
    Args:
        npz_path (str): Path to the input .npz file.
        output_ply_path (str): Path where the .ply will be saved.
        denormalize (bool): If True, uses saved centroid and scale to 
                           restore original car dimensions.
    """
    # 1. Load the NPZ data
    data = np.load(npz_path)
    pts = data['points']      # Shape: (N, 3)
    
    # 2. Denormalize (Optional)
    # This moves the car back from the "unit sphere" to its original CAD coordinates
    if denormalize and 'centroid' in data and 'scale' in data:
        centroid = data['centroid']
        scale = data['scale']
        pts = (pts * scale) + centroid
        print(f"Denormalized using scale: {scale:.4f}")

    # 3. Create Open3D PointCloud object
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts)

    # 4. Save to PLY
    o3d.io.write_point_cloud(output_ply_path, pcd)
    print(f"Successfully saved PLY to: {output_ply_path}")

# --- Example Usage ---
if __name__ == "__main__":
    # Change these to your actual paths
    INPUT_NPZ = "02_dataset/car_4096_pc/pc_processed/F_D_WM_WW_0001.npz"
    OUTPUT_PLY = "pc_car_4098_normalized.ply"

    if os.path.exists(INPUT_NPZ):
        npz_to_ply(INPUT_NPZ, OUTPUT_PLY, denormalize=False)
    else:
        print(f"File {INPUT_NPZ} not found.")

Successfully saved PLY to: pc_car_4098_normalized.ply
